# Homework 6: Finite Element Method using local assembly
Author: Will Eaton \
Last updated: Wednesday 23rd March




## Derivations: 

### Weak form equations
We seek to solve the differential equation 
$$ \rho c_p \partial_t T - \partial_x(\kappa \partial_x T) = f $$
using a finite element method.

First the weak form is obtained by integrating over the domain $x \in [0, L]$ and multiplying by a test function, $w(x)$: 

$$ \int_0^L \rho(x) c_p(x) w(x) \partial_t T(x,t) \: dx - \int_0^L w(x) \partial_x(\kappa(x) \partial_x T(x,t)) \: dx = \int_0^L w(x)f(x) dx $$

Writing the x and t dependences be implicitly, using Gauss' theorem this is rearranged to

$$ \int_0^L \rho c_p w \partial_t T \: dx - \int_0^L  \partial_x(w\kappa \partial_x T) \: dx + \int_0^L  \kappa  (\partial_x w) (\partial_x T) \: dx = \int_0^L wf dx $$

where the second integral can be evaluated knowing that $w(L) = 0$ by construction: 

$$ \int_0^L \rho c_p w \partial_t T \: dx +   w(0)\kappa \partial_x T + \int_0^L  \kappa  (\partial_x w) (\partial_x T) \: dx = \int_0^L wf dx $$

In both problems for this homework, the boundary condition $\kappa \partial_x T(0,t) = -q_0$ hence the weak form becomes 

$$ \int_0^L \rho c_p w \partial_t T \: dx - w(0)q_0 + \int_0^L  \kappa  (\partial_x w) (\partial_x T) \: dx = \int_0^L wf dx. $$

### Mapping to the elemental level

This equation can then be solved by expressing each term as a function of a local elemental domain $\xi \in [-1, +1]$. The mapping for each element is expressed in terms of linear shape functions as: 
$$ x(\xi) = N^1x_A + N^2 x_{A+1} $$ 

where $N^a = \frac{1}{2}(1 + (-1)^a \xi) $, or explicitly

$$ x(\xi) = \frac{h\xi + x_A + x_{A+1}}{2} $$
$$ \xi(x) = \frac{2x - x_A -x_{A+1}}{h} $$

where $h = (x_{A+1} - x_A)$ is the element spacing and $x_A, x_{A+1}$ represent the left and right bounds of the element in x coordinates, for element $A$. For reference the Jacobian and inverse Jacobian are therefore 
$$ J = \frac{\partial x}{\partial \xi} = \frac{h}{2} \:\: ;\:\:\:\: J^{-1} = \frac{\partial \xi}{\partial x} = \frac{2}{h}$$

The weak form can then be written as a summation over all of the $n$ elements in the $\xi$ domain:

$$ \sum_{e=1}^n \int_{-1}^{+1} \rho^e(\xi) c^e_p(\xi)  w^e(\xi)  \partial_t T^e(\xi, t) \: J_e \: d\xi - w^e(-1)q_0 \delta_{1e} + \sum_{e=1}^n \int_{-1}^{+1}  \kappa^e(\xi)  (\partial_{\xi} w^e(\xi)) (\partial_{\xi} T^e(\xi,t))\: J_e^{-1}\: d\xi = \sum_{e=1}^n \int_{-1}^{+1} w^e(\xi) f^e(\xi) \: J d\xi. $$

where the superscript $e$ (subscript on the Jacobians) indicates the element in question. A delta function is introduced to indicate that the boundary term is only applied on the first element. 
Discretisation of the test function and temperature of an element both use the same shape functions (Galerkin method) which are the linear shape functions used for the geometric mapping: 

$$T^e(x(\xi), t) = \sum_{b=1}^2 d^e_b(t) N^b(\xi) + T_1N^{n+1}$$

$$w^e(x(\xi)) = \sum_{a=1}^2 c^e_a N^a(\xi) .$$

The $T_1N^{n+1}$ term is the RHS boundary condition with constant temperature $T_1$. Subbing these in and dropping the explicit dependence on $\xi$ the weak form is discretised as   

$$ \sum_{e=1}^n  \frac{h^e}{2} \int_{-1}^{+1} \rho^e c^e_p  \sum_{b=1}^2 \sum_{a=1}^2 c^e_a N^a N^b   \partial_td^e_b \: d\xi  + \sum_{e=1}^n \frac{2}{h^e} \int_{-1}^{+1}  \kappa^e  \sum_{a=1}^2 \sum_{b=1}^2 c^e_a (\partial_{\xi}N^a)  (\partial_{\xi} N^b) d^e_b \:  d\xi \: = \:  c^e_a N^1q_0 \delta_{1e} \: + \:  \sum_{e=1}^n \frac{h^e}{2} \int_{-1}^{+1} \sum_{a=1}^2  c^e_a N^a f^e \: d\xi - \frac{2}{h^n} \int_{-1}^{+1} \kappa^n (\partial_{\xi}N^{n})  (\partial_{\xi} N^{n+1})T_1 \: d\xi $$

Choosing $c_a^e = 1$ and  defining the following

$$m^e_{ab} = \frac{h^e}{2} \int_{-1}^{+1} \rho^e c^e_p   N^a N^b  \: d\xi$$

$$ k^e_{ab} = \frac{2}{h^e} \int_{-1}^{+1}  \kappa^e \partial_{\xi}N^a \partial_{\xi} N^b \:  d\xi $$ 

$$ f^e_a =\frac{h^e}{2} \int_{-1}^{+1}  N^a f^e \: d\xi $$

we can (mercifully) shorten this equation slightly to  

$$ \sum_{e=1}^n  \sum_{b=1}^2 \sum_{a=1}^2 m^e_{ab} \partial_td^e_b   + \sum_{e=1}^n \sum_{a=1}^2 \sum_{b=1}^2 k^e_{ab} d^e_b \:  = \:  N^1q_0 \delta_{1e} \: + \:  \sum_{e=1}^n \sum_{a=1}^2 f^e_a  \: - \: k^n_{n,\: n+1}T_1 \: d\xi. $$

Knowing that the shape functions are 2-element vectors of $N^1 = [1,0]$ and $N^2 = [0,1]$, It is relatively easy to evaluate the 2D matrices for $m^e$ and $k^e$ assuming that $\kappa, \: \rho$ and $c_p$ remain constant within an element. We can then assemble the whole system into a set of global matrices:

$$ \mathbf{M}\mathbf{\dot{d}} + \mathbf{K}\mathbf{d} = \mathbf{f} + \mathbf{B}$$

where $\mathbf{M}$ is the global mass matrix, $\mathbf{K}$ is the stiffness matrix, $\mathbf{d}$ holds the coefficients to determine temperature. 

### Assembly

The integrals of products of the shape functions and their derivatives are constant so long as we assume constant material properties and force within an element (which is the case for this homework). In this case we find that for any element, $e$, force, mass and stiffness matrices are 

\begin{equation} \mathbf{f}^e = \frac{f h^e}{2}
\begin{bmatrix}
1 \\
1
\end{bmatrix}
\end{equation}

\begin{equation} \mathbf{m}^e = \frac{\rho^e c_p^e h^e}{6}
\begin{bmatrix}
2 & 1 \\
1 & 2
\end{bmatrix}
\end{equation}

\begin{equation} \mathbf{k}^e = \frac{\kappa^e}{h^e}
\begin{bmatrix}
1 & -1 \\
-1 & 1
\end{bmatrix}
\end{equation}


Assembly of the global matrices can be done efficiently for each element by looping through $n$ elements and adding the local matrices as follows:
```
for ni in range(n):
    ni2 = ni+2
    K[ni:ni2, ni:ni2] += k
    F[ni:ni2] += f
    M[ni:ni2, ni:ni2] += m
```

where the lower case indices are the local (2x1 or 2x2) arrays and the capital letters are the global arrays. To do this looping method requires that the global arrays have dimension of (n+1 x 1 or n+1 x n+1). To remove this extra element/row/column at the end we can then just slice the arrays - ensuring that the n+1 element of arrays like the temperature are not affected (by only using the first n (or n x n) elements of the above global arrays) implements the RHS boundary condition as, when looping in time, the velocity is kept at 0 and therefore the last node (array element - the RHS boundary) is not updated/stays fixed.

Finally the vector written above as $\mathbf{B}$ is to implement the boundary conditions and is all zeros except the first and last elements which have values of $q_0$ and $k_{12}T_1$, respectively. 


Marching in time can then be done as per the homework slides. 


## Problem 1: 

Problem 1 is run from the python script ```Problem1.py```. It uses a class called ```Diffusion``` which is stored in ```classes.py```. Initialisation and calculation of the global matrices is all done as part of the constructor. A second function of ```Diffusion``` called ```run()``` does the timestepping and the animation. The resulting animation is shown below: 

In [1]:
from IPython.display import Video
Video("videos/problem1.mp4")


## Problem 2: 

The script for Problem 2 is in ```Problem2.py```. It also uses a class in ```classes.py``` which is called ```Ocean_diffusion```. This class is basically the same as the ```Diffusion``` class (I copied and pasted it!) but with some extra stuff. If I wasn't so behind on other work I would have written them into one single, flexible class or as inherited classes of an ABC sorry! Another small class called ```Isotherm``` is used to help plot some isotherms that I thought would look cool. However, to clearly see the isotherm structure you need to use a lot of elements (~500). This is computationally fine but seems to produce a small instability near the surface which decays through time. This instability is not present when there are fewer elements (e.g 100) or if a smaller timestep is used (e.g. 0.05) which can be set in ```Problem2.py```. So in conclusion I think this is maybe due to having too many elements for a given timestep (which is weird given I thought it would be unconditionally stable), instead of a problem with the boundary condition. 

The second plot on each animation is a number of isotherms (or rather, plotting the y value at which the array index closest to 200$^o C$, 400$^o C$, etc... are for each timestep (iteration). When there are few elements, as in the first two examples, it doesnt really show a clear picture. However using a high number of elements shows the isotherm evolution through time. If we define the base of the lithosphere as some critical temperature, say 600 degrees, then this maps out the lithospheric thickness through time. 

Anyway, here are some examples: 


### Example 1: 
The first example uses 10 elements as per the homework request. It is not a great approximation but the boundary conditions etc are honoured - however if you continue to run the simulation for a very long time then there are discrepencies between the Finite Element and analytical solutions at depth (highest temperatures). I think this is because the analytical solution uses a boundary condition at $x = \infty$ where as the FEM uses a boundary condition at $x = 20$ and hence there are differences through time. 

In [2]:
from IPython.display import Video
Video("videos/problem2_n10_dt_0.01.mp4")

### Example 2: 
The second example uses 100 elements to give a much smoother and more agreeable result. Note here that there is no instability at the surface. 

In [3]:
from IPython.display import Video
Video("videos/problem2_n100_dt_0.01.mp4")

### Example 3: 
The final example uses 500 elements. The reason is that I wanted to plot the isotherms accurately with time (the bottom plot on each of these simulations). If you pause the video at the very start you will see the red dotted line has large negative and positve values close to the surface. This disappears very quickly so isnt really noticable in the simulation but is a weird result!  

In [4]:
from IPython.display import Video
Video("videos/problem2_n500_dt_0.01.mp4")